# Script to evaluate survey responses using LLM



# Configuration
INITIALIZE spreadsheet_file
INITIALIZE questions_list
INITIALIZE llm_api_connection
INITIALIZE survey_data (rows 3 to n)



In [ ]:
#INITIALIZE spreadsheet_file

#INITIALIZE questions_list

In [ ]:
#INITIALIZE llm_api_connection

In [ ]:
#Copy survey_data (rows 3 to n) from survey_spreadsheet to questions_spreadsheet

# Step 1: Populate first row with questions




In [ ]:
#FUNCTION populate_questions():
#    FOR each question IN questions_list:
#        column_index = get_next_available_column()
#        write_to_cell(row=1, col=column_index, value=question)
#    END FOR
#END FUNCTION

# Step 2: Generate LLM answers for each question (row 2)




In [ ]:
# FUNCTION generate_llm_answers():
#     FOR each question IN row 1:
#         column_index = get_column_index(question)
        
#         # Prompt LLM to answer the question
#         prompt = "Answer the following question: " + question
#         llm_answer = call_llm_api(prompt)
        
#         # Write LLM answer to row 2
#         write_to_cell(row=2, col=column_index, value=llm_answer)
#     END FOR
# END FUNCTION

# Step 3: Create evaluation columns




In [ ]:
print("Question grader setup complete.")

: 

In [ ]:
# FUNCTION create_evaluation_columns():
#     question_count = count_columns_with_questions()
    
#     FOR i = 1 TO question_count:
#         # Insert new column after each question column
#         question_col = get_column_index_for_question(i)
#         evaluation_col = question_col + 1
        
#         insert_column(at=evaluation_col)
        
#         # Add header for evaluation column
#         evaluation_header = "Q" + i + "_evaluation"
#         write_to_cell(row=1, col=evaluation_col, value=evaluation_header)
#     END FOR
# END FUNCTION

# Step 4: Evaluate each human answer



In [ ]:
def evaluate_human_answers(worksheet, anthropic_client):
    """
    Evaluate each human answer against the LLM's known answer.
    
    Args:
        worksheet: The worksheet object containing questions and answers
        anthropic_client: Initialized Anthropic client for API calls
    """
    # Get all values from the worksheet
    all_values = worksheet.get_all_values()
    
    if len(all_values) < 3:
        print("Not enough rows to evaluate (need at least 3 rows)")
        return
    
    headers = all_values[0]
    llm_answers = all_values[1]
    
    # Find question columns (columns without "_evaluation" suffix)
    question_columns = []
    for idx, header in enumerate(headers):
        if header and not header.endswith('_evaluation'):
            question_columns.append(idx)
    
    print(f"Found {len(question_columns)} questions to evaluate")
    
    # Iterate through all human responses (rows 3 onwards)
    for row_idx in range(2, len(all_values)):
        row_number = row_idx + 1  # 1-indexed for spreadsheet
        human_responses = all_values[row_idx]
        
        print(f"Evaluating row {row_number}...")
        
        # Evaluate each question for this human
        for question_num, question_col in enumerate(question_columns, start=1):
            question_text = headers[question_col]
            known_answer = llm_answers[question_col]
            
            # Get human answer
            if question_col < len(human_responses):
                human_answer = human_responses[question_col]
            else:
                human_answer = ""
            
            # Skip if human answer is empty
            if not human_answer or human_answer.strip() == "":
                print(f"  Q{question_num}: Skipping (no answer)")
                continue
            
            # Construct evaluation prompt
            evaluation_prompt = f"""Question: {question_text}
Known correct answer: {known_answer}
Human answer to evaluate: {human_answer}

Please evaluate the human answer compared to the known answer.
Provide a review including:
- Accuracy assessment
- Key points covered or missed
- Overall quality rating"""
            
            try:
                # Get LLM evaluation using Anthropic API
                message = anthropic_client.messages.create(
                    model="claude-3-5-sonnet-20241022",
                    max_tokens=1024,
                    messages=[
                        {"role": "user", "content": evaluation_prompt}
                    ]
                )
                
                evaluation_result = message.content[0].text
                
                # Write evaluation to the column right after the question column
                evaluation_col = question_col + 1
                # Convert to A1 notation (1-indexed)
                cell_address = f"{chr(65 + evaluation_col)}{row_number}"
                worksheet.update(cell_address, evaluation_result)
                
                print(f"  Q{question_num}: Evaluated and written to {cell_address}")
                
            except Exception as e:
                error_message = f"Error evaluating: {str(e)}"
                print(f"  Q{question_num}: {error_message}")
                # Write error to evaluation column
                evaluation_col = question_col + 1
                cell_address = f"{chr(65 + evaluation_col)}{row_number}"
                worksheet.update(cell_address, error_message)
    
    print("Human answer evaluation complete!")

# Main execution


In [ ]:
# FUNCTION main():
#     # Step 1: Set up questions
#     populate_questions()
    
#     # Step 2: Get LLM answers
#     generate_llm_answers()
    
#     # Step 3: Create evaluation columns
#     create_evaluation_columns()
    
#     # Step 4: Evaluate all human answers
#     evaluate_human_answers()
    
#     # Save spreadsheet
#     save_spreadsheet()
    
#     PRINT "Evaluation complete!"
# END FUNCTION

# # Run the script
# main()